In [1]:
import os
import sys
import importlib

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

import configs
importlib.reload(configs)

from pyspark.sql import SparkSession

print("configs.py em:", configs.__file__)
print("JDBC_URL:", configs.JDBC_URL)
print("BRONZE_PATH:", configs.BRONZE_PATH)


configs.py em: c:\Users\viami\Documents\projetos\teste-tecnico-engenharia-de-dados-sicredi-pl\configs.py
JDBC_URL: jdbc:sqlserver://localhost:1433;databaseName=sicredi;encrypt=false;trustServerCertificate=true;
BRONZE_PATH: c:\Users\viami\Documents\projetos\teste-tecnico-engenharia-de-dados-sicredi-pl\data\bronze


In [2]:
spark = (
    SparkSession.builder
    .appName("sicredi-bronze")
    .config("spark.jars.packages", configs.JDBC_JAR_PACKAGE)
    .getOrCreate()
)

spark


In [3]:
connection_props = {
    "user": configs.DB_USER,
    "password": configs.DB_PASSWORD,
    "driver": configs.JDBC_DRIVER,
}

def read_table(table_name: str):
    print(f"Lendo tabela {table_name} do SQL Server...")
    return spark.read.jdbc(
        url=configs.JDBC_URL,
        table=table_name,
        properties=connection_props,
    )


In [ ]:
df_associado = read_table("dbo.associado")
df_associado.show(5)


Lendo tabela dbo.associado do SQL Server...
+---+-----------+-----------+-----+--------------------+
| id|       nome|  sobrenome|idade|               email|
+---+-----------+-----------+-----+--------------------+
|  1|      Joana|     Vargas|   49|ana-julia31@examp...|
|  2|     Camila|    Rezende|   44|cirinomaria-cecil...|
|  3|    Bárbara|    Pimenta|   44|santosmatteo@exam...|
|  4|Luiz Otávio|Vasconcelos|   75|mouraeloah@exampl...|
|  5|      Diogo|     Farias|   18|maysaguerra@examp...|
+---+-----------+-----------+-----+--------------------+
only showing top 5 rows


In [8]:
df_associado = read_table("dbo.associado")
df_conta = read_table("dbo.conta")
df_cartao = read_table("dbo.cartao")
df_mov = read_table("dbo.movimento")

df_associado.show(5)


Lendo tabela dbo.associado do SQL Server...
Lendo tabela dbo.conta do SQL Server...
Lendo tabela dbo.cartao do SQL Server...
Lendo tabela dbo.movimento do SQL Server...
+---+-----------+-----------+-----+--------------------+
| id|       nome|  sobrenome|idade|               email|
+---+-----------+-----------+-----+--------------------+
|  1|      Joana|     Vargas|   49|ana-julia31@examp...|
|  2|     Camila|    Rezende|   44|cirinomaria-cecil...|
|  3|    Bárbara|    Pimenta|   44|santosmatteo@exam...|
|  4|Luiz Otávio|Vasconcelos|   75|mouraeloah@exampl...|
|  5|      Diogo|     Farias|   18|maysaguerra@examp...|
+---+-----------+-----------+-----+--------------------+
only showing top 5 rows


In [9]:
(df_associado.write.mode("overwrite")
 .parquet(configs.BRONZE_PATH + "/associado"))

(df_conta.write.mode("overwrite")
 .parquet(configs.BRONZE_PATH + "/conta"))

(df_cartao.write.mode("overwrite")
 .parquet(configs.BRONZE_PATH + "/cartao"))

(df_mov.write.mode("overwrite")
 .parquet(configs.BRONZE_PATH + "/movimento"))
